In [1]:
import os
import sys
import pickle
import matplotlib
import numpy as np
import pandas as pd
# import seaborn as sns

from matplotlib import pyplot as plt
from scipy.spatial.transform import Rotation as R

from argparse import ArgumentParser

from filelock import FileLock
from pprint import pprint
from PIL import Image

matplotlib.use('TkAgg')

curr_dir = os.getcwd()
if curr_dir != '/home/mulerod1/projects/dynamic_dual_manip/':
    os.chdir('/home/mulerod1/projects/dynamic_dual_manip/')
print(f"Current working directory is {os.getcwd()}")

from bcm.manipulation_utils import generate_fling_trajectory, generate_quasi_static_trajectory, compute_angle

import open3d as o3d
from bcm.vision.utils import chamfer_distance_w_unidirectional, pcu_unidirectional_chamfer_distance
from bcm.utils import load_targets
import torch
import point_cloud_utils as pcu


Current working directory is /home/mulerod1/projects/dynamic_dual_manip
Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [ ]:
# Plot per task
from matplotlib import rcParams

# plt.style.use('seaborn')
plt.style.use('seaborn-white')
# rcParams['figure.figsize'] = 11.7, 8.27
rcParams['figure.figsize'] = 6, 6
rcParams['figure.dpi'] = 200
rcParams['figure.subplot.left'] = 0.1
rcParams['figure.subplot.right'] = 0.99
rcParams['figure.subplot.bottom'] = 0.01
rcParams['figure.subplot.top'] = 0.99
rcParams['axes.grid'] = False
# rcParams['grid.color'] = "#ffffff"
rcParams['grid.color'] = "#000000"
rcParams['grid.linewidth'] = 1.0
rcParams['grid.linestyle'] = ":"
# X ticks
rcParams['xtick.major.bottom'] = True
rcParams['xtick.bottom'] = True
rcParams['xtick.minor.visible'] = True
rcParams['xtick.minor.size'] = 5.5
rcParams['xtick.minor.width'] = 1.0
# Y ticks
rcParams['ytick.major.left'] = True
rcParams['ytick.left'] = True
rcParams['ytick.minor.visible'] = True
rcParams['ytick.minor.size'] = 5.5
rcParams['ytick.minor.width'] = 1.0
rcParams['ytick.major.left'] = True
#ytick.major.size:    3.5     # major tick size in points
#ytick.labelsize:     medium  # font size of the tick labels
rcParams['ytick.labelsize'] = 'large'
rcParams['xtick.labelsize'] = "large"

# Font
# rcParams['font.family'] = "serif"
# rcParams['font.style'] = "normal"
rcParams['text.usetex'] = True
rcParams['ps.fonttype'] = 42

# Lines
rcParams['lines.linewidth'] = 0
rcParams['lines.markersize'] = 6

font_value = 41  # Use 50 for the variance plots
# font_value = 100  # Use 50 for the variance plots

rcParams['font.size'] = font_value
# Set specific values for the font relative to font.size
# ## small, medium, large, x-large, xx-large, larger, or smaller
rcParams['axes.titleweight'] = "bold"
# rcParams['axes.labelsize'] = "x-large"
# rcParams['legend.fontsize'] = "xx-large"
# rcParams['legend.title_fontsize'] = 100

In [9]:
def trunc(values, decs=0):
    return np.trunc(values*10**decs)/(10**decs)
nr_decimals=3

In [59]:
def get_target_and_sim_cloud(sim_cloud_traj, timestep, target_tgt):
    
    # print(f"Sim = {sim} shape source = {sim_cloud_traj.shape[0]}")
    
    # Define the source to target match 
    # target_it = np.linspace(0, sim_cloud_traj.shape[0], len(target_tgt), dtype=int)
    
    skip_frames = 0
    
    # Load the target
    freq_relation = sim_cloud_traj.shape[0]/len(target_tgt)
    
    iteration_nr = int(timestep  * freq_relation)
    
    # target_idx = int(target_it[iteration_nr] - skip_frames)
    target_idx = int(iteration_nr/freq_relation - skip_frames)
    print(f"Freq relation={freq_relation}, iteration = {iteration_nr} target idx={target_idx}")

    target_cloud = torch.from_numpy(target_tgt[target_idx]).unsqueeze(0).float()
    
    target_vertices = target_cloud.numpy()[0]
    
    # Select the source cloud to match
    # source_nr = target_it[iteration_nr]
    sim_cloud_npy = sim_cloud_traj[iteration_nr]
    sim_cloud_torch = torch.from_numpy(sim_cloud_npy).unsqueeze(0)
    
    # print(f"Shape sim = {sim_cloud_torch.shape} taret={transformed_target.shape}")
    cd_cost_l1 = chamfer_distance_w_unidirectional(sim_cloud_torch.type(torch.FloatTensor), target_cloud.type(torch.FloatTensor), single_directional=True, norm=1)[0]
    if sim_cloud_npy.shape[0] < target_vertices.shape[0]:
        first_sim_cloud = target_vertices.copy()
        first_sim_cloud[:sim_cloud_npy.shape[0], :] = sim_cloud_npy
        first_sim_cloud[sim_cloud_npy.shape[0]:, :] = np.nan
        sim_cloud_npy = first_sim_cloud[~np.isnan(first_sim_cloud[:, 0]), :]
    else:
        first_sim_cloud = sim_cloud_npy.copy()
        first_sim_cloud[:target_vertices.shape[0], :] = target_vertices
        first_sim_cloud[target_vertices.shape[0]:, :] = np.nan
        target_vertices = first_sim_cloud[~np.isnan(first_sim_cloud[:, 0]), :]
    hd_p1_to_p2 = pcu.one_sided_hausdorff_distance(sim_cloud_npy, target_vertices, return_index=False)
    # print(f"Sim = {sim}, nr points = {sim_cloud_npy.shape[0]}, target points={target_vertices.shape[0]}")
    # print(f"Sim = {sim}, cd_cost_l1 = {cd_cost_l1}, hd_p1_to_p2={hd_p1_to_p2}, iteration = {iteration_nr} target idx={target_idx}")
    
    # Reduce the number of points so that they can be plotted
    source_temp = o3d.geometry.PointCloud()
    source_temp.points = o3d.utility.Vector3dVector(sim_cloud_npy)
    downpcd = source_temp.voxel_down_sample(voxel_size=0.02)
    
    min_voxel_size = 0.025
    voxel_step = 0.005
    target_shape = 500
    
    if np.asarray(downpcd.points).shape[0] > target_shape:
        downpcd = downpcd.voxel_down_sample(voxel_size=min_voxel_size+voxel_step)
    
    sim_cloud_reduced = np.asarray(downpcd.points)
    
    target_temp = o3d.geometry.PointCloud()
    target_temp.points = o3d.utility.Vector3dVector(target_vertices)
    # 0.03 500 - 1k, 1k4 
    # 0.02 = 1-2k points, 2k
    # 0.01 = ~5-6k
    
    target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size)
    if np.asarray(target_downpcd.points).shape[0] > target_shape:
        target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step)
        if np.asarray(target_downpcd.points).shape[0] > target_shape:
            target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step*2)
        if np.asarray(target_downpcd.points).shape[0] > target_shape:
                target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step*3)
                if np.asarray(target_downpcd.points).shape[0] > target_shape:
                    target_downpcd = target_temp.voxel_down_sample(voxel_size=min_voxel_size+voxel_step*4)
        
    
    target_cloud_reduced = np.asarray(target_downpcd.points)
    
    # print(f"Reduced cloud sim nr points = {sim_cloud_reduced.shape[0]}, target points={target_cloud_reduced.shape[0]}")
    
    
    # return target_cloud_reduced, sim_cloud_reduced, cd_cost_l1, hd_p1_to_p2
    return target_cloud_reduced, sim_cloud_npy, cd_cost_l1, hd_p1_to_p2


In [62]:
def get_alphas(sim_name):
    if sim_name == 'mujoco3':
        # alpha_red = 0.5  # Looks good
        alpha_red = 0.08
        alpha_black = 0.05
    elif sim_name == 'bullet':
        alpha_red = 0.08  # Maybe a bit less?
        # alpha_red = 0.15  # looks ok-ish Maybe a bit less?
        alpha_black = 0.05
    elif sim_name == 'softgym':
        # alpha_red = 0.01 # not too bad, a bit too much
        # alpha_red = 0.05 # too much
        alpha_red = 0.01
        alpha_black = 0.05
    else: # sofa
        alpha_red = 0.05
        alpha_black = 0.05
        
    return alpha_red, alpha_black
    
    
def plot_primitive_rag(sim_folders, times_to_plot, source_dir, primitive, rag, seed, elev=8, azim=13):
    
    data_path = "/home/mulerod1/projects/dynamic_dual_manip/dataset/point_clouds"
    target_path = f"{data_path}/{rag}/{primitive}/cloud/" 
    target_tgt = load_targets(target_path)
    
    width=10*len(times_to_plot)
    height = 10

    fig, ax = plt.subplots(4, len(times_to_plot), figsize=(width,height), subplot_kw=dict(projection='3d'), dpi=300)
    i = 0
    
    if primitive == 'quasi_static':
        real_time = np.around(np.linspace(start=0.0, stop=14, num=420), decimals=2)
    else:  # dynamic
        real_time = np.around(np.linspace(start=0.0, stop=4.4, num=129), decimals=2)
        
    for sim in sim_folders:
        j=0
        alpha_red, alpha_black = get_alphas(sim)
        
        for time in times_to_plot:
            
            
            idx = (np.abs(real_time - time)).argmin()
            # print(f'idx={idx}, timevalue={real_time[idx]}')
            skip_frames = 0
            target_idx = idx + skip_frames
            
            print(f"Sim={sim} time={time} idx={idx} target_idx={target_idx}")
            source_path = source_dir + sim + f'_{rag}_{primitive}_{seed}_cloth_sequence.npy'
        
            sim_cloud_traj = np.load(source_path)
            (target_vertices, temp_sim_cloud_npy,
             cd_cost_l1, hd_p1_to_p2) = get_target_and_sim_cloud(sim_cloud_traj, target_idx, target_tgt)
            
            # Check what is the maximum Z point index, which are the actual grippers 
            if sim == 'bullet' and primitive == "quasi_static":
                idx_max = np.argmax(temp_sim_cloud_npy[:, 2])
                # print(f"Max_idx value={idx_max}, Z={temp_sim_cloud_npy[idx_max, 2]}")
                temp_2_cloud = np.delete(temp_sim_cloud_npy, idx_max, axis=0)
                # what is the second largest?
                idx_max2 = np.argmax(temp_2_cloud[:, 2])
                # print(f"Max_idx value={idx_max2}, Z={temp_2_cloud[idx_max2, 2]}")
                sim_cloud_npy = np.delete(temp_2_cloud, idx_max2, axis=0)
            else:
                sim_cloud_npy = temp_sim_cloud_npy
            
            ax[i, j].scatter(sim_cloud_npy[:, 0], sim_cloud_npy[:, 1], sim_cloud_npy[:, 2],
                       marker='*', color='r', label='Sim', alpha=alpha_red)
            
            ax[i, j].scatter(target_vertices[:, 0], target_vertices[:, 1], target_vertices[:, 2],
                       marker='^', color='k', label='Real', alpha=alpha_black)
            
            ax[i, j].text(0.5, -0.2, -0.3, s=f'CD={trunc(np.round(cd_cost_l1.item(), decimals=nr_decimals), decs=nr_decimals)}', fontsize=12)
                    # verticalalignment='bottom', horizontalalignment='right',
                    # transform=ax.transAxes)
            # ax.text(0.5, 0.5, 0.5, s=f'CD={trunc(cd_cost_l1)}', fontsize=15,
            #         verticalalignment='bottom', horizontalalignment='right',
            #         transform=ax.transAxes)
            # Write HD
            ax[i, j].text(0.5, -0.2, -0.5, s=f'HD={trunc(np.round(hd_p1_to_p2, decimals=nr_decimals), decs=nr_decimals)}', fontsize=12)
            
            # ax[i].set_title(sim)
            ax[i, j].grid(visible=False)
            ax[i, j].view_init(elev=elev, azim=azim)
            if primitive == 'quasi_static':
                ax[i, j].set_xlim([-0.1, 0.6])
                ax[i, j].set_ylim([-0.7, 0.1])
                ax[i, j].set_zlim([0.00, 1.1])
            else:
                ax[i, j].set_xlim([-0.4, 0.4])
                ax[i, j].set_ylim([-0.8, 0.8])
                ax[i, j].set_zlim([0.00, 1.1])
            ax[i, j].set_yticklabels([])
            ax[i, j].set_xticklabels([])
            ax[i, j].set_aspect('equal')
            # ax[i, j].set_xticks([])
            # ax[i, j].set_yticks([])
            # plt.gca().axes.get_yaxis().set_visible(False)
            # ax[i,j ].set_axis_off()
            
            j+=1
        
        i+=1
        # plt.suptitle(f'Sim={sim}')
    # plt.subplots_adjust(wspace=0, hspace=0)
    
    plt.savefig(f'comparison_clouds_{primitive}.pdf')
    # plt.show()

In [72]:
%matplotlib inline
# Real, t=0 timestep=11, t=4.4 timestep=131

sim_folders = ['mujoco3', 'bullet', 'softgym', 'sofa']
# sim_folders = ['mujoco3', 'bullet']


# target = "cotton_rag_1"  # Bullet wrong
# target = "chequered_rag_1"
target = "linen_rag_1"
limit_quasi_static = 420  # 14 seconds
limit_dynamic = 98  # 129 = 4.4 seconds, 98=3.62 seconds

source_dir = "/home/mulerod1/projects/dynamic_dual_manip/results_npy_27/"





quasi = 'quasi_static'

dyn = 'dynamic'



# real_time = np.around(np.linspace(start=0.0, stop=4.4, num=120), decimals=2)

# times_to_plot = [0.7, 1.0, 1.4, 2.0, 2.5, 3.0, 3.5, 4.0]
# times_to_plot = [0.0, 0.7, 1.0, 1.4, 2.0, 2.5, 3.0, 3.5, 4.4]
# times_to_plot = [0.5, 1.0, 1.5, 2.0, 2.5, 3.0, 3.5, 4.4]
# second 1, Z touches, second 2 waiting, second 2-8 moves down to Zmin
# second 8-14, moves to Xmin
times_to_plot_quasi = [1.0, 4.0, 8.0, 14.0]
times_to_plot_dyn = [0.5, 1.5, 2.5, 3.5]


In [ ]:
plot_primitive_rag(sim_folders=sim_folders,
                   times_to_plot=times_to_plot_quasi, 
                   source_dir=source_dir,
                   primitive=quasi, 
                   rag=target, seed=1050,
                   elev=8, azim=18)


In [73]:
plot_primitive_rag(sim_folders=sim_folders,
                   times_to_plot=times_to_plot_dyn, 
                   source_dir=source_dir,
                   primitive=dyn, 
                   rag=target, seed=1050,
                   elev=8, azim=18)

Sim=mujoco3 time=0.5 idx=14 target_idx=14
Freq relation=3.4, iteration = 47 target idx=13
Sim=mujoco3 time=1.5 idx=44 target_idx=44
Freq relation=3.4, iteration = 149 target idx=43
Sim=mujoco3 time=2.5 idx=73 target_idx=73
Freq relation=3.4, iteration = 248 target idx=72
Sim=mujoco3 time=3.5 idx=102 target_idx=102
Freq relation=3.4, iteration = 346 target idx=101
Sim=bullet time=0.5 idx=14 target_idx=14
Freq relation=3.4, iteration = 47 target idx=13
Sim=bullet time=1.5 idx=44 target_idx=44
Freq relation=3.4, iteration = 149 target idx=43
Sim=bullet time=2.5 idx=73 target_idx=73
Freq relation=3.4, iteration = 248 target idx=72
Sim=bullet time=3.5 idx=102 target_idx=102
Freq relation=3.4, iteration = 346 target idx=101
Sim=softgym time=0.5 idx=14 target_idx=14
Freq relation=3.4, iteration = 47 target idx=13
Sim=softgym time=1.5 idx=44 target_idx=44
Freq relation=3.4, iteration = 149 target idx=43
Sim=softgym time=2.5 idx=73 target_idx=73
Freq relation=3.4, iteration = 248 target idx=72
